# Лабораторная работа №6

## Подготовка входных данных

In [77]:
!pip install catboost

  Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
Using cached catboost-1.2.7-cp312-cp312-win_amd64.whl (101.7 MB)
Using cached graphviz-0.20.3-py3-none-any.whl (47 kB)


In [79]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [81]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

In [82]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

In [85]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

In [87]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

In [89]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [95]:
import numpy as np

from catboost import CatBoostClassifier

In [97]:
cat_boost_classifier = CatBoostClassifier()
cat_boost_params ={
    "iterations": range(1,6),
    'learning_rate': [0.1,0.25,0.5,0.75,1],
    'depth':range(1,6),
}
cat_boost_grid = GridSearchCV(cat_boost_classifier, cat_boost_params, cv=2, n_jobs=1,verbose=3)
cat_boost_grid.fit(X_train, y_train)
best_cat_boost_params = cat_boost_grid.best_params_

Fitting 2 folds for each of 125 candidates, totalling 250 fits
0:	learn: 0.6513390	total: 147ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.1;, score=0.820 total time=   0.4s
0:	learn: 0.6508832	total: 2.27ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.1;, score=0.814 total time=   0.2s
0:	learn: 0.5968842	total: 2.51ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.25;, score=0.820 total time=   0.2s
0:	learn: 0.5958351	total: 3.85ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.25;, score=0.814 total time=   0.3s
0:	learn: 0.5271776	total: 2.26ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.5;, score=0.820 total time=   0.2s
0:	learn: 0.5253405	total: 2.3ms	remaining: 0us
[CV 2/2] END depth=1, iterations=1, learning_rate=0.5;, score=0.814 total time=   0.2s
0:	learn: 0.4814982	total: 2.27ms	remaining: 0us
[CV 1/2] END depth=1, iterations=1, learning_rate=0.75;, score=0.820 tot

In [98]:
best_des_tree_model = CatBoostClassifier()
best_des_tree_model.fit(X_train, y_train)
des_tree_predicted = best_des_tree_model.predict(X_test)
print('Used params:',best_cat_boost_params)
print('Evaluation:\n', metrics.classification_report(y_test, des_tree_predicted,digits=5))

Learning rate set to 0.042157
0:	learn: 0.6633874	total: 13.9ms	remaining: 13.9s
1:	learn: 0.6353085	total: 28.3ms	remaining: 14.1s
2:	learn: 0.6092720	total: 42ms	remaining: 14s
3:	learn: 0.5863317	total: 56.3ms	remaining: 14s
4:	learn: 0.5656573	total: 71.2ms	remaining: 14.2s
5:	learn: 0.5474040	total: 87.6ms	remaining: 14.5s
6:	learn: 0.5299162	total: 102ms	remaining: 14.5s
7:	learn: 0.5148157	total: 117ms	remaining: 14.6s
8:	learn: 0.5001775	total: 133ms	remaining: 14.6s
9:	learn: 0.4881968	total: 148ms	remaining: 14.6s
10:	learn: 0.4765163	total: 162ms	remaining: 14.6s
11:	learn: 0.4664481	total: 176ms	remaining: 14.5s
12:	learn: 0.4578234	total: 189ms	remaining: 14.3s
13:	learn: 0.4486815	total: 202ms	remaining: 14.2s
14:	learn: 0.4404489	total: 218ms	remaining: 14.3s
15:	learn: 0.4334096	total: 233ms	remaining: 14.3s
16:	learn: 0.4269868	total: 247ms	remaining: 14.3s
17:	learn: 0.4210494	total: 260ms	remaining: 14.2s
18:	learn: 0.4162064	total: 273ms	remaining: 14.1s
19:	learn: 